Collecting data from Yfinance

In [2]:
import pandas as pd
import yfinance as yf

# Define the list of tickers and the S&P 500 index
tickers = ['AAPL', 'AMZN', 'BRK-B', 'GOOGL', 'JNJ', 'JPM', 'META', 'MSFT', 'NVDA', 'TSLA']
index_ticker = '^GSPC'  # S&P 500 index ticker

# Define the start and end dates
start_date = '2019-01-02'
end_date = '2023-12-29'

# Initialize an empty DataFrame to hold all the stock data
all_stock_data = pd.DataFrame()

# Fetch historical stock data
for ticker in tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Ticker'] = ticker
    stock_data['Date'] = stock_data.index
    stock_data.reset_index(drop=True, inplace=True)
    all_stock_data = pd.concat([all_stock_data, stock_data])

# Fetch historical S&P 500 data
index_data = yf.download(index_ticker, start=start_date, end=end_date)
index_data['Date'] = index_data.index
index_data.reset_index(drop=True, inplace=True)
index_data.rename(columns={'Close': 'Index_Close'}, inplace=True)

# Merge stock data with index data on Date
merged_data = pd.merge(all_stock_data, index_data[['Date', 'Index_Close']], on='Date', how='inner')

# Select relevant columns and arrange them
final_data = merged_data[['Date', 'Ticker', 'Close', 'Index_Close']]

# Save the final data to a CSV file
final_data.to_csv('stock_with_index_data.csv', index=False)

print("Data combined and saved to CSV in the desired format.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data combined and saved to CSV in the desired format.


### Checking the data for errors such as missing values, duplicate rows, and outliers. 

In [3]:
import pandas as pd

# Load the dataset
file_path = 'stock_with_index_data.csv'
data = pd.read_csv(file_path)

# 1. Check for missing values
missing_values = data.isnull().sum()
print("Missing values per column:\n", missing_values)

# 2. Check for duplicate rows
duplicate_rows = data.duplicated().sum()
print("Number of duplicate rows: ", duplicate_rows)

# 3. Check for outliers
# Here, we use the IQR method to detect outliers in numeric columns
def detect_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outlier_indices = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))].index
        outliers[col] = outlier_indices
    return outliers

outliers = detect_outliers(data)
print("Outliers detected in the following columns:\n", {k: len(v) for k, v in outliers.items()})

# 4. Check data types
data_types = data.dtypes
print("Data types of each column:\n", data_types)

# Save the cleaned dataset
cleaned_file_path = 'cleaned_stock_with_index_data.csv'
data.to_csv(cleaned_file_path, index=False)

print("Cleaned dataset saved to:", cleaned_file_path)


Missing values per column:
 Date           0
Ticker         0
Close          0
Index_Close    0
dtype: int64
Number of duplicate rows:  0
Outliers detected in the following columns:
 {'Close': 54, 'Index_Close': 0}
Data types of each column:
 Date            object
Ticker          object
Close          float64
Index_Close    float64
dtype: object
Cleaned dataset saved to: cleaned_stock_with_index_data.csv


### Addressing outliers using the IQR (Interquartile Range) method:

In [4]:
import pandas as pd

# Load the cleaned dataset
data = pd.read_csv('cleaned_stock_with_index_data.csv')

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['Close'].quantile(0.25)
Q3 = data['Close'].quantile(0.75)
IQR = Q3 - Q1

# Define the bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify and remove outliers
outliers = data[(data['Close'] < lower_bound) | (data['Close'] > upper_bound)]
print("Outliers detected and removed:\n", outliers)

# Remove outliers
data_cleaned = data[(data['Close'] >= lower_bound) & (data['Close'] <= upper_bound)]

# Save the cleaned data without outliers
data_cleaned.to_csv('cleaned_stock_with_index_data_no_outliers.csv', index=False)

print("Cleaned dataset without outliers saved to: cleaned_stock_with_index_data.csv")


Outliers detected and removed:
              Date Ticker       Close  Index_Close
6456   2021-07-26   META  372.459991  4422.299805
6476   2021-07-28   META  373.279999  4400.640137
6696   2021-08-27   META  372.630005  4509.370117
6706   2021-08-30   META  380.660004  4528.790039
6716   2021-08-31   META  379.380005  4522.680176
6726   2021-09-01   META  382.049988  4524.089844
6736   2021-09-02   META  375.279999  4536.950195
6746   2021-09-03   META  376.260010  4535.430176
6756   2021-09-07   META  382.179993  4520.029785
6766   2021-09-08   META  377.570007  4514.069824
6776   2021-09-09   META  378.000000  4493.279785
6786   2021-09-10   META  378.690002  4458.580078
6796   2021-09-13   META  376.510010  4468.729980
6806   2021-09-14   META  376.529999  4443.049805
6816   2021-09-15   META  373.920013  4480.700195
6826   2021-09-16   META  373.059998  4473.750000
7139   2021-10-29   TSLA  371.333344  4605.379883
7149   2021-11-01   TSLA  402.863342  4613.669922
7159   2021-11-02 

#### Getting data for testing:

In [6]:
import pandas as pd
import yfinance as yf

# Define the list of tickers
tickers = ['AAPL', 'AMZN', 'BRK-B', 'GOOGL', 'JNJ', 'JPM', 'META', 'MSFT', 'NVDA', 'TSLA']
index_ticker = '^GSPC'  # S&P 500 Index

# Define the date range
start_date = '2024-01-01'
end_date = '2024-07-31'

# Initialize an empty DataFrame to hold all the stock data
all_stock_data = pd.DataFrame()

# Loop through each ticker and fetch the data
for ticker in tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Ticker'] = ticker
    stock_data.reset_index(inplace=True)
    all_stock_data = pd.concat([all_stock_data, stock_data], axis=0)

# Download the S&P 500 index data
index_data = yf.download(index_ticker, start=start_date, end=end_date)
index_data.reset_index(inplace=True)
index_data.rename(columns={'Close': 'Index_Close'}, inplace=True)

# Merge stock data with index data on Date
merged_data = pd.merge(all_stock_data, index_data[['Date', 'Index_Close']], on='Date', how='inner')

# Select relevant columns and arrange them
final_data = merged_data[['Date', 'Ticker', 'Close', 'Index_Close']]

# Save the data to a CSV file
final_data.to_csv('new_stock_with_index_data.csv', index=False)

print("New stock data with index saved to: new_stock_with_index_data.csv")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


New stock data with index saved to: new_stock_with_index_data.csv
